# 17.3 Data Cleaning 2: Missing Values
1. Determine all the variable types and find the fraction of the missing values for each variable.
2. Notice that the data has a time dimension (year). For this assignment, forget about time and treat all the observations as if they're from the same year. Choose a strategy to deal with the missing values for each variables. For which variables would filling in the missing values with some value make sense? For which might tossing out the records entirely make sense?
3. Now, take into account the time factor. Replicate your second answer but this time fill in the missing values by using a statistic that is calculated within the year of the observation. For example, if you want to fill a missing value for a variable with the mean of that variable, calculate the mean by using only the observations for that specific year.
4. This time, fill in the missing values using interpolation (extrapolation).
5. Compare your results for the 2nd, 3rd, and 4th questions. Do you find any meaningful differences?

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
                    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

df = pd.read_sql_query('select * from useducation', con=engine)

engine.dispose()
df.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.327876,252.187522,207.963517,NaN
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,...,9748.0,8789.0,6714.0,79117.0,30847.0,112335.0,NaN,NaN,NaN,258.859712
2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,...,55433.0,49081.0,37410.0,437127.0,175210.0,614881.0,215.253932,265.366278,206.212716,262.169895
3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,...,34632.0,36011.0,27651.0,281338.0,123113.0,405259.0,210.206028,256.312090,208.634458,264.619665
4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,...,418418.0,363296.0,270675.0,3286034.0,1372011.0,4717112.0,208.398961,260.892247,196.764414,NaN


### 1. The variable types are as follows:

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1492 entries, 0 to 1491
Data columns (total 25 columns):
PRIMARY_KEY                     1492 non-null object
STATE                           1492 non-null object
YEAR                            1492 non-null int64
ENROLL                          1229 non-null float64
TOTAL_REVENUE                   1280 non-null float64
FEDERAL_REVENUE                 1280 non-null float64
STATE_REVENUE                   1280 non-null float64
LOCAL_REVENUE                   1280 non-null float64
TOTAL_EXPENDITURE               1280 non-null float64
INSTRUCTION_EXPENDITURE         1280 non-null float64
SUPPORT_SERVICES_EXPENDITURE    1280 non-null float64
OTHER_EXPENDITURE               1229 non-null float64
CAPITAL_OUTLAY_EXPENDITURE      1280 non-null float64
GRADES_PK_G                     1319 non-null float64
GRADES_KG_G                     1360 non-null float64
GRADES_4_G                      1361 non-null float64
GRADES_8_G                      1

### Percentage of missing values
Note that the average scores is missing a huge chunk of data at 64% while the rest of the data is between 8~18%.

In [4]:
df.isna().sum() / df.isna().count()

PRIMARY_KEY                     0.000000
STATE                           0.000000
YEAR                            0.000000
ENROLL                          0.176273
TOTAL_REVENUE                   0.142091
FEDERAL_REVENUE                 0.142091
STATE_REVENUE                   0.142091
LOCAL_REVENUE                   0.142091
TOTAL_EXPENDITURE               0.142091
INSTRUCTION_EXPENDITURE         0.142091
SUPPORT_SERVICES_EXPENDITURE    0.142091
OTHER_EXPENDITURE               0.176273
CAPITAL_OUTLAY_EXPENDITURE      0.142091
GRADES_PK_G                     0.115952
GRADES_KG_G                     0.088472
GRADES_4_G                      0.087802
GRADES_8_G                      0.087802
GRADES_12_G                     0.087802
GRADES_1_8_G                    0.087802
GRADES_9_12_G                   0.087802
GRADES_ALL_G                    0.115952
AVG_MATH_4_SCORE                0.640751
AVG_MATH_8_SCORE                0.643432
AVG_READING_4_SCORE             0.642761
AVG_READING_8_SC

## Central Tendency
Looking at the data the mean is skewed higher than the median of the data. Meaning that there might be some clear outliers in the data. Note that in this assignment the outliers will not be explored. As such the median will be used for the fills below. 

In [5]:
round(df.describe(),2)

,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
count,1492.00,1229.00,1280.00,1280.00,1280.00,1280.00,1280.00,1280.00,1280.00,1229.00,...,1361.00,1361.00,1361.00,1361.00,1361.00,1319.00,536.00,532.00,533.00,498.00
mean,2004.43,915930.82,9092081.86,766372.34,4216552.95,4109156.57,9196680.87,4762966.37,2680330.68,429204.62,...,64538.93,64271.06,54268.92,519214.04,247071.35,802441.51,234.77,278.41,218.87,263.66
std,7.39,1065280.42,11745187.40,1145241.59,5543072.00,5482970.61,11978126.90,6293003.72,3353348.56,534069.32,...,80293.39,78910.55,67889.28,643638.43,307705.63,970703.07,10.22,10.16,7.77,6.80
min,1992.00,43866.00,465650.00,31020.00,0.00,22093.00,481665.00,265549.00,139963.00,11541.00,...,633.00,437.00,311.00,4878.00,1808.00,7254.00,187.13,232.83,178.56,236.38
25%,1998.00,258314.00,2186305.25,189354.00,1153097.25,715834.50,2165404.50,1168032.00,635790.00,102831.00,...,13739.00,13342.00,10919.00,110626.00,51471.00,181065.00,229.69,272.76,214.66,259.53
50%,2004.00,648313.00,5079546.00,403376.50,2537073.50,2055780.50,5234505.50,2657452.00,1525405.50,271596.00,...,43272.00,43339.00,36735.00,345775.00,164260.00,550342.00,237.24,280.62,220.42,265.01
75%,2011.00,1014528.00,10859847.75,828966.00,5080939.25,4768680.00,10745191.00,5568028.00,3229650.75,518600.00,...,75481.00,76566.00,67460.00,611905.00,290502.00,928275.50,242.00,285.35,224.00,268.19
max,2017.00,6307022.00,89217262.00,9990221.00,50904567.00,36105265.00,85320133.00,43964520.00,26058021.00,3995951.00,...,493415.00,500143.00,498403.00,3929869.00,2013687.00,5944746.00,253.42,300.57,236.77,280.50


### 2. 
The median for the population was used to fill the entire dataframe. The issue with this is that the data in the 1992 year is higher than the year 1990s years. This fill will skew the data to a higher amount. The benefit of this fill is that it removes all the NAN items. 

In [6]:
col_list = ['ENROLL', 'TOTAL_REVENUE', 'FEDERAL_REVENUE', 'STATE_REVENUE', 
            'LOCAL_REVENUE', 'TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE', 
            'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE', 'CAPITAL_OUTLAY_EXPENDITURE', 
            'GRADES_PK_G', 'GRADES_KG_G', 'GRADES_4_G', 'GRADES_8_G', 'GRADES_12_G', 'GRADES_1_8_G',
           'GRADES_9_12_G', 'GRADES_ALL_G', 'AVG_MATH_4_SCORE', 'AVG_MATH_8_SCORE', 'AVG_READING_4_SCORE',
           'AVG_READING_8_SCORE']

df_fill_1 = df.copy()
df_fill_1[col_list] = df_fill_1[col_list].apply(lambda x: x.fillna(x.mean()))
df_fill_1

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,915930.820993,2.678885e+06,3.041770e+05,1.659028e+06,7.156800e+05,2.653798e+06,1.481703e+06,...,57948.000000,58025.000000,41167.00000,4.715640e+05,1.963860e+05,6.761740e+05,208.327876,252.187522,207.963517,263.661132
1,1992_ALASKA,ALASKA,1992,915930.820993,1.049591e+06,1.067800e+05,7.207110e+05,2.221000e+05,9.724880e+05,4.983620e+05,...,9748.000000,8789.000000,6714.00000,7.911700e+04,3.084700e+04,1.123350e+05,234.768293,278.414711,218.866154,258.859712
2,1992_ARIZONA,ARIZONA,1992,915930.820993,3.258079e+06,2.978880e+05,1.369815e+06,1.590376e+06,3.401580e+06,1.435908e+06,...,55433.000000,49081.000000,37410.00000,4.371270e+05,1.752100e+05,6.148810e+05,215.253932,265.366278,206.212716,262.169895
3,1992_ARKANSAS,ARKANSAS,1992,915930.820993,1.711959e+06,1.785710e+05,9.587850e+05,5.746030e+05,1.743022e+06,9.643230e+05,...,34632.000000,36011.000000,27651.00000,2.813380e+05,1.231130e+05,4.052590e+05,210.206028,256.312090,208.634458,264.619665
4,1992_CALIFORNIA,CALIFORNIA,1992,915930.820993,2.626002e+07,2.072470e+06,1.654651e+07,7.641041e+06,2.713883e+07,1.435892e+07,...,418418.000000,363296.000000,270675.00000,3.286034e+06,1.372011e+06,4.717112e+06,208.398961,260.892247,196.764414,263.661132
5,1992_COLORADO,COLORADO,1992,915930.820993,3.185173e+06,1.632530e+05,1.307986e+06,1.713934e+06,3.264826e+06,1.642466e+06,...,50648.000000,45025.000000,34533.00000,3.949040e+05,1.602990e+05,5.626130e+05,221.023429,272.398433,213.480890,266.419687
6,1992_CONNECTICUT,CONNECTICUT,1992,915930.820993,3.834302e+06,1.435420e+05,1.342539e+06,2.348221e+06,3.721338e+06,2.148041e+06,...,38058.000000,33691.000000,28366.00000,3.042840e+05,1.269170e+05,4.369320e+05,226.798480,273.739345,222.417599,264.713904
7,1992_DELAWARE,DELAWARE,1992,915930.820993,6.452330e+05,4.594500e+04,4.209420e+05,1.783460e+05,6.387840e+05,3.727220e+05,...,8272.000000,8012.000000,6129.00000,6.749500e+04,2.833800e+04,9.629600e+04,217.899972,262.868585,206.389052,261.577921
8,1992_DISTRICT_OF_COLUMBIA,DISTRICT_OF_COLUMBIA,1992,915930.820993,7.094800e+05,6.474900e+04,0.000000e+00,6.447310e+05,7.428930e+05,3.291600e+05,...,5832.000000,5000.000000,3433.00000,4.700900e+04,1.817300e+04,7.000000e+04,192.600553,234.924621,178.557612,265.976765
9,1992_FLORIDA,FLORIDA,1992,915930.820993,1.150630e+07,7.884200e+05,5.683949e+06,5.033930e+06,1.130564e+07,5.166374e+06,...,164416.000000,142372.000000,100835.00000,1.276685e+06,5.115570e+05,1.819706e+06,213.689751,259.908848,204.882606,262.118986


In [7]:
round(df_fill_1.groupby('YEAR').median(), 2)

,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
YEAR,,,,,,,,,,,,,,,,,,,,,
1992,915930.82,3132831.50,239119.50,1404433.00,1414301.00,3198671.00,1599618.00,976780.50,429204.62,231450.50,...,45118.00,41886.50,32700.50,356393.50,150913.00,574791.50,221.89,271.64,218.87,263.66
1993,642550.50,3204929.50,221567.00,1481432.50,1424718.50,3217803.50,1652333.00,976003.00,177862.00,208558.00,...,44686.50,42984.00,32719.00,358588.50,155165.00,577969.50,234.77,278.41,218.87,263.66
1994,642984.50,3357720.00,248698.50,1551033.50,1503780.50,3365686.50,1708475.00,1014688.00,186794.00,253627.50,...,44834.00,43811.00,33717.00,359977.00,155307.50,578854.50,234.77,278.41,218.87,263.66
1995,683736.00,3750835.00,272138.00,1830162.50,1555362.50,3750897.00,1893385.50,1097246.00,219839.50,278666.00,...,44277.50,45537.50,33883.50,361916.50,156911.00,584839.50,234.77,278.41,218.87,263.66
1996,735912.00,3891203.00,287181.00,1967170.00,1698978.00,4075165.00,2052264.00,1139539.00,228647.00,367884.00,...,41969.00,42862.00,34795.00,337917.00,157020.00,568065.00,225.81,276.77,218.87,263.66
1997,737386.00,4133275.00,303240.00,2087903.00,1836068.00,4398196.00,2188731.00,1205308.00,246858.00,439273.00,...,43175.00,42466.00,35900.00,340533.00,159395.00,566158.00,234.77,278.41,218.87,263.66
1998,756941.00,4559382.00,326866.00,2267168.50,2014359.00,4732617.00,2320735.00,1310402.50,264621.50,481893.00,...,43652.00,41539.50,35392.50,343253.50,156609.50,567702.50,234.77,278.41,218.87,263.66
1999,764642.00,5059587.00,372016.00,2369846.00,2271871.00,5072471.00,2547187.00,1426620.00,271812.00,608284.00,...,45307.00,42721.00,36594.00,351202.00,159788.00,580292.00,234.77,278.41,218.87,263.66
2000,750755.00,5338531.00,409021.00,2481089.00,2446818.00,5444193.00,2601684.00,1573568.00,300044.00,716110.00,...,43436.00,42364.00,33911.00,338900.00,155734.00,576743.00,230.57,278.41,218.87,263.66


In [8]:
df_fill_1.isna().sum() / df_fill_1.isna().count()

PRIMARY_KEY                     0.0
STATE                           0.0
YEAR                            0.0
ENROLL                          0.0
TOTAL_REVENUE                   0.0
FEDERAL_REVENUE                 0.0
STATE_REVENUE                   0.0
LOCAL_REVENUE                   0.0
TOTAL_EXPENDITURE               0.0
INSTRUCTION_EXPENDITURE         0.0
SUPPORT_SERVICES_EXPENDITURE    0.0
OTHER_EXPENDITURE               0.0
CAPITAL_OUTLAY_EXPENDITURE      0.0
GRADES_PK_G                     0.0
GRADES_KG_G                     0.0
GRADES_4_G                      0.0
GRADES_8_G                      0.0
GRADES_12_G                     0.0
GRADES_1_8_G                    0.0
GRADES_9_12_G                   0.0
GRADES_ALL_G                    0.0
AVG_MATH_4_SCORE                0.0
AVG_MATH_8_SCORE                0.0
AVG_READING_4_SCORE             0.0
AVG_READING_8_SCORE             0.0
dtype: float64

## 3. 
For question 3 we will fill the missing data based on the year enrolled. The big issue with this method is that if all the variables in a year is NAN then it will return NAN. The benefit is that the median is more accurate than step 2 above. 

In [9]:
df_fill_2 = df.copy()

# Note that this will apply the fill by the year group and keep the data disaggreagated
df_fill_2 = df_fill_2.groupby('YEAR', as_index=False).apply(lambda x: x.fillna(x.mean())).reset_index()
df_fill_2.head()

,level_0,level_1,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,0,0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,...,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.327876,252.187522,207.963517,263.307067
1,0,1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,...,9748.0,8789.0,6714.0,79117.0,30847.0,112335.0,218.410116,266.360319,212.712256,258.859712
2,0,2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,...,55433.0,49081.0,37410.0,437127.0,175210.0,614881.0,215.253932,265.366278,206.212716,262.169895
3,0,3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,...,34632.0,36011.0,27651.0,281338.0,123113.0,405259.0,210.206028,256.312090,208.634458,264.619665
4,0,4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,...,418418.0,363296.0,270675.0,3286034.0,1372011.0,4717112.0,208.398961,260.892247,196.764414,263.307067


In [10]:
tmp2 = df.groupby('YEAR').sum(min_count=1)
tmp2

,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
YEAR,,,,,,,,,,,,,,,,,,,,,
1992,NaN,233210984.0,15148508.0,106504038.0,111558438.0,237333143.0,125154024.0,69877598.0,NaN,20655398.0,...,3400593.0,3184852.0,2466171.0,27201911.0,11647686.0,39243672.0,9173.224882,11187.133399,8508.490223,2106.456535
1993,40904771.0,243206654.0,15739744.0,112674291.0,114792619.0,244202408.0,129561976.0,72413365.0,12659974.0,20890815.0,...,3418482.0,3305113.0,2460085.0,27531479.0,11894650.0,39860586.0,NaN,NaN,NaN,NaN
1994,40870582.0,250750573.0,16688390.0,113508868.0,120553315.0,252664869.0,133930062.0,75321616.0,13408752.0,20611622.0,...,3482346.0,3357384.0,2524041.0,27812202.0,12151319.0,40455192.0,NaN,NaN,NaN,NaN
1995,43807625.0,279277709.0,18085196.0,130307103.0,130885410.0,281831835.0,149485905.0,82430356.0,15061557.0,24304092.0,...,3486494.0,3410794.0,2523428.0,28116955.0,12435462.0,41080164.0,NaN,NaN,NaN,NaN
1996,44570461.0,292952786.0,18604617.0,139602106.0,134746063.0,297740499.0,156920519.0,85717821.0,15479323.0,27771656.0,...,3517427.0,3460749.0,2625092.0,28668642.0,12836473.0,42068120.0,9745.098392,11077.483652,8583.481970,9933.659692
1997,45307367.0,311885259.0,19736543.0,149946399.0,142202317.0,315861220.0,165523905.0,89870126.0,15569010.0,32434149.0,...,3571049.0,3471642.0,2711798.0,28935947.0,13034172.0,42559915.0,NaN,NaN,NaN,NaN
1998,45772962.0,331730773.0,21789839.0,160178264.0,149762670.0,336384794.0,174817066.0,95572187.0,16266761.0,35873565.0,...,3661346.0,3541144.0,2762384.0,29258081.0,13184965.0,43111630.0,NaN,NaN,NaN,NaN
1999,46160175.0,354681711.0,24214071.0,172904755.0,157562885.0,360474429.0,185292396.0,101981137.0,17071405.0,40480233.0,...,3756708.0,3558656.0,2824926.0,29469229.0,13375118.0,43540093.0,NaN,NaN,NaN,NaN
2000,46432782.0,379602220.0,26672292.0,186191266.0,166738654.0,386177409.0,198033848.0,108952594.0,17969490.0,44654247.0,...,3778713.0,3600838.0,2845177.0,29736804.0,13550123.0,43927538.0,9216.944550,10921.144183,9574.954814,11295.553564


In [11]:
df_fill_2.isna().sum() / df_fill_2.isna().count()

level_0                         0.000000
level_1                         0.000000
PRIMARY_KEY                     0.000000
STATE                           0.000000
YEAR                            0.000000
ENROLL                          0.071716
TOTAL_REVENUE                   0.034182
FEDERAL_REVENUE                 0.034182
STATE_REVENUE                   0.034182
LOCAL_REVENUE                   0.034182
TOTAL_EXPENDITURE               0.034182
INSTRUCTION_EXPENDITURE         0.034182
SUPPORT_SERVICES_EXPENDITURE    0.034182
OTHER_EXPENDITURE               0.071716
CAPITAL_OUTLAY_EXPENDITURE      0.034182
GRADES_PK_G                     0.068365
GRADES_KG_G                     0.068365
GRADES_4_G                      0.068365
GRADES_8_G                      0.068365
GRADES_12_G                     0.068365
GRADES_1_8_G                    0.068365
GRADES_9_12_G                   0.068365
GRADES_ALL_G                    0.068365
AVG_MATH_4_SCORE                0.576408
AVG_MATH_8_SCORE

## 4.
Interpolate populated all the NAN values unlike step 3. One issue could be the outliers skewing the data. 

In [12]:
df_fill_3 = df.copy()

df_fill_3[col_list] = df_fill_3[col_list].apply(lambda x: x.interpolate(method='linear', limit_direction='forward'))#.reset_index(drop=True)
df_fill_3[col_list] = df_fill_3[col_list].apply(lambda x: x.interpolate(method='linear', limit_direction='backward'))
df_fill_3[col_list] = df_fill_3[col_list].apply(lambda x: x.interpolate(method='nearest'))
round(df_fill_3, 2).head()


,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,89711.0,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.33,252.19,207.96,258.86
1,1992_ALASKA,ALASKA,1992,89711.0,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,...,9748.0,8789.0,6714.0,79117.0,30847.0,112335.0,211.79,258.78,207.09,258.86
2,1992_ARIZONA,ARIZONA,1992,89711.0,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,...,55433.0,49081.0,37410.0,437127.0,175210.0,614881.0,215.25,265.37,206.21,262.17
3,1992_ARKANSAS,ARKANSAS,1992,89711.0,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,...,34632.0,36011.0,27651.0,281338.0,123113.0,405259.0,210.21,256.31,208.63,264.62
4,1992_CALIFORNIA,CALIFORNIA,1992,89711.0,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,...,418418.0,363296.0,270675.0,3286034.0,1372011.0,4717112.0,208.40,260.89,196.76,265.52


In [13]:
df_fill_3.isna().sum() / df_fill_3.isna().count()

PRIMARY_KEY                     0.0
STATE                           0.0
YEAR                            0.0
ENROLL                          0.0
TOTAL_REVENUE                   0.0
FEDERAL_REVENUE                 0.0
STATE_REVENUE                   0.0
LOCAL_REVENUE                   0.0
TOTAL_EXPENDITURE               0.0
INSTRUCTION_EXPENDITURE         0.0
SUPPORT_SERVICES_EXPENDITURE    0.0
OTHER_EXPENDITURE               0.0
CAPITAL_OUTLAY_EXPENDITURE      0.0
GRADES_PK_G                     0.0
GRADES_KG_G                     0.0
GRADES_4_G                      0.0
GRADES_8_G                      0.0
GRADES_12_G                     0.0
GRADES_1_8_G                    0.0
GRADES_9_12_G                   0.0
GRADES_ALL_G                    0.0
AVG_MATH_4_SCORE                0.0
AVG_MATH_8_SCORE                0.0
AVG_READING_4_SCORE             0.0
AVG_READING_8_SCORE             0.0
dtype: float64

## 5. 
For this question all the above items will be compared. The folllowing graphs compares each of the fills above. What was interesting is that the interpolated items had a higher mean than the items that filled in questions 2 and 3. Whereas the median was clearly higher for all the items. It was surprising that overall the data did not change drastically. Out of all the choses the filled values by year was more accurate but had large NAN populations, followed by the interpolated items. 

In [14]:
for col in col_list:
    print("Statistics for columns: {}".format(col))
    print(pd.concat([df[col], df_fill_1[col],
                     df_fill_2[col], df_fill_3[col]], axis=1).describe())

Statistics for columns: ENROLL
             ENROLL        ENROLL        ENROLL        ENROLL
count  1.229000e+03  1.492000e+03  1.385000e+03  1.492000e+03
mean   9.159308e+05  9.159308e+05  9.161654e+05  9.838759e+05
std    1.065280e+06  9.667725e+05  1.003532e+06  1.006773e+06
min    4.386600e+04  4.386600e+04  4.386600e+04  4.386600e+04
25%    2.583140e+05  3.150940e+05  2.867590e+05  2.821195e+05
50%    6.483130e+05  8.204140e+05  7.374010e+05  7.373935e+05
75%    1.014528e+06  9.211780e+05  9.624880e+05  1.595024e+06
max    6.307022e+06  6.307022e+06  6.307022e+06  6.307022e+06
Statistics for columns: TOTAL_REVENUE
       TOTAL_REVENUE  TOTAL_REVENUE  TOTAL_REVENUE  TOTAL_REVENUE
count   1.280000e+03   1.492000e+03   1.441000e+03   1.492000e+03
mean    9.092082e+06   9.092082e+06   9.084108e+06   1.111164e+07
std     1.174519e+07   1.087818e+07   1.110204e+07   1.200155e+07
min     4.656500e+05   4.656500e+05   4.656500e+05   4.656500e+05
25%     2.186305e+06   2.546261e+06   2.425

          GRADES_8_G     GRADES_8_G     GRADES_8_G     GRADES_8_G
count    1361.000000    1492.000000    1390.000000    1492.000000
mean    64271.057311   64271.057311   64331.744106   59511.380027
std     78910.545776   75364.301721   78083.976028   77123.573668
min       437.000000     437.000000     437.000000     437.000000
25%     13342.000000   15682.750000   13698.250000   10252.000000
50%     43339.000000   49571.000000   44756.500000   38114.500000
75%     76566.000000   72131.000000   75016.250000   72598.250000
max    500143.000000  500143.000000  500143.000000  500143.000000
Statistics for columns: GRADES_12_G
        GRADES_12_G    GRADES_12_G    GRADES_12_G    GRADES_12_G
count    1361.00000    1492.000000    1390.000000    1492.000000
mean    54268.92432   54268.924320   54418.075582   50207.239276
std     67889.27623   64838.328605   67187.337112   66309.867155
min       311.00000     311.000000     311.000000     311.000000
25%     10919.00000   13437.000000   11161.75